# 1.0 Importing Libraries

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from matplotlib import pyplot as plt

In [2]:
pd.set_option('display.max_columns', 50)

# 2.0 Loading Data

In [3]:
df_sales_group = pd.read_csv('dataset/sales_group.csv')

author = pd.read_csv('dataset/author.csv')
award = pd.read_csv('dataset/award.csv')
book = pd.read_csv('dataset/book.csv')
edition = pd.read_csv('dataset/edition.csv')
format = pd.read_csv('dataset/format.csv')
genders = pd.read_csv('dataset/genders.csv')
info = pd.read_csv('dataset/info.csv')
publisher = pd.read_csv('dataset/publisher.csv')
ratings = pd.read_csv('dataset/ratings.csv')
series = pd.read_csv('dataset/series.csv')

### 2.1 Basic Data Transformation

In [4]:
# Renaming the columns on the award dataset to eliminate spaces
cols_new = ['book_id', 'title', 'award_name', 'year_won']
award.columns = cols_new

In [5]:
# Checking for duplicates
datasets = [author, award, book, edition, format, info, publisher, series, genders, ratings]
datasets_names = ['author', 'award', 'book', 'edition', 'format', 'info', 'publisher', 'series', 'genders', 'ratings']

i=0
for dataset in datasets:
    print(f'Dataset: {datasets_names[i]} - Quantidade de duplicadas: {dataset.duplicated().sum()}\n')
    i+=1

Dataset: author - Quantidade de duplicadas: 0

Dataset: award - Quantidade de duplicadas: 0

Dataset: book - Quantidade de duplicadas: 0

Dataset: edition - Quantidade de duplicadas: 0

Dataset: format - Quantidade de duplicadas: 0

Dataset: info - Quantidade de duplicadas: 0

Dataset: publisher - Quantidade de duplicadas: 119

Dataset: series - Quantidade de duplicadas: 0

Dataset: genders - Quantidade de duplicadas: 0

Dataset: ratings - Quantidade de duplicadas: 0



In [6]:
# Lets remove the duplicates from the publisher dataset
publisher.drop_duplicates(inplace=True)

### 2.2 Merging the datasets

In [7]:
df = pd.merge(book, author, how='left', on='author_id')
df = pd.merge(df, info, how='left', on='book_id')
df = pd.merge(df, series, how='left', on='series_id')
df = pd.merge(df, genders, how='left', on='genre_id')

#### 2.2.1 Award dataset

The award dataset has duplicated book_ids for books with multiple awards, lets aggregate them before merging

In [8]:
award_agg = award.groupby('book_id').agg({
    'title': 'first',
    'award_name': lambda x: ', '.join(x),
    'year_won': 'first'}).reset_index()

df = pd.merge(df, award_agg, how='left', on='book_id')

#### 2.2.2 Publisher dataset

The publisher dataset also has duplicated book_ids for books with multiple publishers. But, we have a problem that some publishers have two different pub_ids, so we can't merge them yet.

So, first we'll determine that the right pub_id is the one who corresponds to the initials of the publisher name.


In [9]:
# Defining useful functions

def get_initials(name):
    return "".join(word[0] for word in name.split()).upper()

def select_pub_id(group):
    initials = group["initials"].iloc[0]
    matching_pub_id = group[group["pub_id"] == initials]

    if not matching_pub_id.empty:
        return matching_pub_id.iloc[0]
    else:
        return group.sort_values("pub_id").iloc[0]


In [10]:
# Getting the initials of the publisher name
publisher["initials"] = publisher["name"].apply(get_initials)

# Selecting the correct pub_id
publisher_corrected = publisher.groupby(["book_id", "name"], group_keys=False).apply(select_pub_id)
publisher_corrected.reset_index(drop=True, inplace=True)

# Dropping the initial column
publisher_corrected = publisher_corrected.drop(columns=["initials"])
publisher = publisher.drop(columns=["initials"])

Now we can aggregate and merge the datasets.

Here, we'll opt for summing the marketing spend, since it's the corresponding value for each book

In [11]:
# Aggregating the publisher dataset
publisher_agg = publisher_corrected.groupby('book_id').agg({
    'name': lambda x: ', '.join(x),
    'city': lambda x: ', '.join(x),
    'state': 'first',
    'country': 'first',
    'year_established': 'first',
    'marketing_spend': 'sum',
    'pub_id': lambda x: ', '.join(x)
}).reset_index()

In [12]:
# Merging the datasets
df = pd.merge(df, publisher_agg, how='left', on='book_id')

#### 2.2.3 Ratings dataset

Let's keep only the average rating and the number of ratings for each book

In [13]:
# Getting the average rating
ratings_mean = ratings.groupby('book_id')['rating'].mean().round(2).reset_index()
ratings_mean.columns = ['book_id', 'rating_mean']

In [14]:
# Getting the number of ratings
ratings_count = ratings.groupby('book_id')['rating'].count().reset_index()
ratings_count.columns = ['book_id', 'rating_count']

In [15]:
# Merging the datasets
df = pd.merge(df, ratings_mean, how='left', on='book_id')
df = pd.merge(df, ratings_count, how='left', on='book_id')

#### 2.2.4 Edition dataset

Lets first merge the edition and the format datasets.

In [16]:
df_edition = pd.merge(edition, format, how='left', on='format_id')

In [17]:
df_edition.drop(columns=['format_id'], inplace=True)

#### 2.2.5 Sales dataset

In [18]:
sales = pd.merge(df_sales_group, df_edition, how='left', on='isbn')
# sales = pd.merge(sales, df, how='left', on=['book_id', 'pub_id'])

# 3.0 Data Transformation

Lets organize the data before merging into the sales dataset

In [19]:
# Getting the author name
df['author_name'] = df['first_name'] + ' ' + df['last_name']

## 3.1 Dataset columns

In [20]:
# Remove unuseful columns
df.drop(columns=['title_y', 'author_id', 'genre_id', 'series_id', 'first_name', 'last_name'], inplace=True)

In [21]:
# Renaming the columns
df.rename(columns={
    'title_x': 'title',
    'genre_desc': 'genre',
    'name': 'pub_name',
    'city': 'pub_city',
    'state': 'pub_state',
    'country': 'pub_country'
}, inplace=True)

In [22]:
sales.rename(columns={
    'COUNT': 'sales_count',
    'format_desc': 'format'
}, inplace=True)

## 3.2 Data types

In [23]:
# Checking the data types
df.dtypes

book_id               object
title                 object
birthday              object
country_residence     object
hrs_writing_day      float64
volume_number        float64
series_name           object
genre                 object
award_name            object
year_won             float64
pub_name              object
pub_city              object
pub_state             object
pub_country           object
year_established     float64
marketing_spend      float64
pub_id                object
rating_mean          float64
rating_count         float64
author_name           object
dtype: object

In [24]:
sales.dtypes

sale_date            object
isbn                 object
sales_count           int64
book_id              object
pub_id               object
publication_date     object
pages                 int64
print_run_size_k      int64
price               float64
format               object
dtype: object

In [25]:
# Transform from object to datetime
df['birthday'] = pd.to_datetime(df['birthday'], format='%d/%m/%Y')
sales['sale_date'] = pd.to_datetime(sales['sale_date'], format='%d/%m/%Y')
sales['publication_date'] = pd.to_datetime(sales['publication_date'], format='%d/%m/%Y')

In [26]:
# Transform from float to int
df['volume_number'] = df['volume_number'].astype('Int64')
df['year_won'] = df['year_won'].astype('Int64')
df['year_established'] = df['year_established'].astype('Int64')
df['marketing_spend'] = df['marketing_spend'].astype('Int64')
df['rating_count'] = df['rating_count'].astype('Int64')
sales['print_run_size_k'] = sales['print_run_size_k'].astype('Int64')
sales['pages'] = sales['pages'].astype('Int64')

## 3.3 Null values

In [27]:
# Checking for missing values
df.isnull().sum()

book_id               0
title                 0
birthday              0
country_residence     0
hrs_writing_day       0
volume_number        38
series_name          38
genre                 0
award_name           41
year_won             41
pub_name              1
pub_city              1
pub_state             1
pub_country           1
year_established      1
marketing_spend       1
pub_id                1
rating_mean           2
rating_count          2
author_name           0
dtype: int64

In [28]:
sales.isnull().sum()

sale_date           0
isbn                0
sales_count         0
book_id             0
pub_id              0
publication_date    0
pages               0
print_run_size_k    0
price               0
format              0
dtype: int64

In [29]:
# volume_number null values are the ones that don't have series
df['volume_number'].fillna(0, inplace=True)

# series_name null values are the ones that also don't have series
df['series_name'].fillna('', inplace=True)

# award_name and year_won null values are the ones that don't have awards, so let's keep them as NaN

# rating_count null values are the ones that don't have ratings
df['rating_count'].fillna(0, inplace=True)

# We have one book that doesn't have a publisher
# Let's keep the rating_mean null value as NaN to not affect the average rating


## 3.4 Check duplicates

In [30]:
df.duplicated().sum()

0

In [31]:
df['book_id'].duplicated().sum()

0

In [32]:
sales.duplicated().sum()

0

## 3.5 Creating sales by date per book dataset

In [33]:
print(f'The df dataset has information about {df.shape[0]} books and the following {df.shape[1]} columns:\n{list(i for i in df.columns)}')

The df dataset has information about 58 books and the following 20 columns:
['book_id', 'title', 'birthday', 'country_residence', 'hrs_writing_day', 'volume_number', 'series_name', 'genre', 'award_name', 'year_won', 'pub_name', 'pub_city', 'pub_state', 'pub_country', 'year_established', 'marketing_spend', 'pub_id', 'rating_mean', 'rating_count', 'author_name']


In [34]:
print(f'The sales dataset has information about {sales.shape[0]} sales and the following {sales.shape[1]} columns:\n{list(i for i in sales.columns)}')

The sales dataset has information about 15188 sales and the following 10 columns:
['sale_date', 'isbn', 'sales_count', 'book_id', 'pub_id', 'publication_date', 'pages', 'print_run_size_k', 'price', 'format']


Now we can finally merge the two datasets.

In [35]:
df_sales = pd.merge(sales, df, how='left', on=['book_id', 'pub_id'])

## 3.6 Creating sales per book dataset

In [36]:
# Create dataset with the total sales for each book
sales_per_book = df_sales.groupby('isbn').agg(total_sales=('sales_count', 'sum')).reset_index()

In [37]:
# Create dataset with the books info
book_info = df_sales.drop(columns=['sale_date', 'sales_count']).drop_duplicates()

In [38]:
# Merge the two datasets
sales_total = pd.merge(sales_per_book, book_info, how='left', on='isbn')

In [39]:
# Create a column for the total revenue for each book
sales_total['total_revenue'] = sales_total['total_sales'] * sales_total['price']

# 4.0 Exploratory Data Analysis

## 4.1 Analysing big numbers

Total invoice

In [40]:
value_to_display = sales_total['total_revenue'].sum()

# Convert value to thousands and format as USD
value_in_k = value_to_display / 1_000

In [41]:
# Create an indicator chart
fig = go.Figure(go.Indicator(
    mode="number", 
    value=value_in_k,
    title={"text": "Total Invoice"},
    number={"prefix": "USD ", "suffix": "K", "font": {"size": 48}},
    domain={"x": [0, 1], "y": [0, 1]}
))

# Update layout for better scaling
fig.update_layout(
    height=250,
    width=400,
    template='plotly_white'
)

# Show the chart
fig.show()

Total sales

In [42]:
value_to_display = sales_total['total_sales'].sum()

# Convert value to thousands
value_in_k = value_to_display / 1_000

# Create an indicator chart
fig = go.Figure(go.Indicator(
    mode="number", 
    value=value_in_k,
    title={"text": "Total Sales Count"},
    number={"prefix": "", "suffix": "K", "font": {"size": 48}},
    domain={"x": [0, 1], "y": [0, 1]}
))

# Update layout for better scaling
fig.update_layout(
    height=250,
    width=400,
    template='plotly_white'
)

# Show the chart    
fig.show()

Number of unique book titles

In [43]:
value_to_display = sales_total['title'].nunique()

# Create an indicator chart
fig = go.Figure(go.Indicator(
    mode="number", 
    value=value_to_display,
    title={"text": "Unique Book Titles"},
    number={"prefix": "", "font": {"size": 48}},
    domain={"x": [0, 1], "y": [0, 1]}
))

# Update layout for better scaling 
fig.update_layout(
    height=250,
    width=400,
    template='plotly_white'
)

# Show the chart
fig.show()

## 4.2 Overview

In [44]:
from plot_utils import create_horizontal_bar_plot, create_vertical_bar_plot, create_box_plot

### 4.2.1 Genre distribution

In [45]:
create_vertical_bar_plot(sales_total, 'genre')

### 4.2.2 Publisher distribution

In [46]:
create_vertical_bar_plot(sales_total, 'pub_name', 'Publisher distribution')

We note that there are only 4 publishers in the dataset and the one with the highest amount of books is Etaoin Shrdlu Press (ESP)

### 4.2.3 Format distribution

In [47]:
create_horizontal_bar_plot(sales_total, 'format')

The most common book format is Hardcover.

### 4.2.4 Author distribution

In [48]:
create_horizontal_bar_plot(sales_total, 'author_name', 'Books per author distribution')

### 4.2.5 Price distribution

In [49]:
price_distribution = sales_total.price.describe()

In [50]:
print(f'The average price of a book is ${price_distribution["mean"]:.2f} and the standard deviation is ${price_distribution["std"]:.2f}')
print(f'The minimum price is ${price_distribution["min"]:.2f} and the maximum price is ${price_distribution["max"]:.2f}')

The average price of a book is $16.84 and the standard deviation is $8.47
The minimum price is $5.99 and the maximum price is $48.00


In [51]:
create_box_plot(sales_total, 'price')

We have only one outlier with a price of $48.

### 4.2.6 Pages distribution

In [52]:
pages_distribution = sales_total.pages.describe()
print(f'The average number of pages is {pages_distribution["mean"].round(0)} and the standard deviation is {pages_distribution["std"]:.2f}')
print(f'The minimum number of pages is {pages_distribution["min"].round(0)} and the maximum number of pages is {pages_distribution["max"].round(0)}')

The average number of pages is 387.0 and the standard deviation is 252.85
The minimum number of pages is 11.0 and the maximum number of pages is 1296.0


In [53]:
create_box_plot(sales_total, 'pages')

### 4.2.7 Geographical distribution

In [54]:
create_horizontal_bar_plot(sales_total, 'country_residence', "Author's country of residence distribution")

In [55]:
# Let's plot the country distribution for the publishers in the dataset
df_plot = sales_total.groupby('pub_country').agg(
    Count=('pub_country', 'count'),
    Publishers=('pub_name', lambda x: ", ".join(sorted(set(x))))
).reset_index()

df_plot.columns = ['pub_country'.replace('_', ' ').title(), 'Count', 'Publishers']
df_plot = df_plot.sort_values('Count', ascending=False)

fig = px.bar(
    df_plot,
    x=df_plot.columns[0],
    y=df_plot.columns[1],
    text=df_plot["Publishers"],
    title=f"Publisher's country distribution",
    color_discrete_sequence=['seagreen']
)

fig.update_traces(textposition="outside")
fig.update_layout(
    template='plotly_white',
    title_font_size=16,
    xaxis_title=df_plot.columns[0],
    yaxis_title='Count',
    height=600,
    width=800
)

fig.show()

## 4.3 Top Sellers

In [56]:
from plot_utils import create_horizontal_high_value_bar_plot, create_vertical_high_value_bar_plot

### 4.3.2 Best-selling and most profitable books

In [60]:
create_horizontal_high_value_bar_plot(sales_total, 'title', 'total_sales', quantile=0.9)

In [61]:
create_horizontal_high_value_bar_plot(sales_total, 'title', 'total_revenue', quantile=0.9)

We see that our top 3 🏆 Best-selling books are:

1. The Mallemaroking
2. Rystwyth
3. Heliotrope Pajamas

And our 💵 Most Profitable books are:

1. Rystwyth
2. The Mallemaroking
3. Portmeirion

# Tests

In [58]:
sales_total.head(1)

,isbn,total_sales,book_id,pub_id,publication_date,pages,print_run_size_k,price,format,title,birthday,country_residence,hrs_writing_day,volume_number,series_name,genre,award_name,year_won,pub_name,pub_city,pub_state,pub_country,year_established,marketing_spend,rating_mean,rating_count,author_name,total_revenue
0,989-28-229-0197-6,502,TO369,SSC,2185-02-08,255,20,9.5,Mass market paperback,The Scent of Oranges,2155-08-19,New Zealand,8.0,4,Esme s Ladies,Romance,NaN,<NA>,Sound & Seas Co.,Dunedin,Otago,New Zealand,1973,151000,4.17,1204,Lynne Danticat,4769.0
